# Tools

This notebook contains all the tools that will be used by the agent.

In [1]:
import requests
import sqlite3
import pandas as pd

from smolagents import tool, Tool

from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun

## Creating simple tool

A Python function must be annotated with `@tool`. It should also have a docstring describing what does the function do, what does it return and the description of its parameters.

### City to location 

The following function look up at latitude and longitude of a city.

In [ ]:
# TODO: Load CSV file containing latitude, longitude and altitude of cities
# https://github.com/bahar/WorldCityLocations/tree/master
df = pd.read_csv('data/cities_latlng.csv', sep=';')
print(df)

In [ ]:
# Add columns to the data frame
df.columns = [ 'id', 'country', 'city', 'latitude', 'longitude', 'altitude']
# Drop country column
df.drop('country', axis=1, inplace=True)

In [5]:
df.head()

,id,city,latitude,longitude,altitude
0,2,Kandahar,31.610000,65.699997,1015
1,3,Mazar-e Sharif,36.706944,67.112221,369
2,4,Herat,34.340000,62.189999,927
3,5,Jalalabad,34.420000,70.449997,573
4,6,Konduz,36.720000,68.860001,394


In [6]:
# TODO: Explore the loaded dataframe
df[df['city'] == 'Singapore']

,id,city,latitude,longitude,altitude
10566,10568,Singapore,1.29027,103.851959,164


In [15]:
# TODO: Add tool description

@tool
def get_latlng(city: str) -> any:
   """ 
   Get the latitude, longitude and altitude of a city in a dictionary. 

   Args:
      city: city name that you want the latitude, longitude and altitude
      
   Returns:
      any: a dictionary with the following keys: city, latitude, longitude and altitude

   Example:
      result = get_latlng('tokyo')   
   """
   r = df.query(f"city.str.lower() == '{city.lower()}'")
   return { 'city': city, 'latitude': r.iloc[0]['latitude'], 'longitude': r.iloc[0]['longitude'], 'altitude': r.iloc[0]['altitude'] }

In [11]:
# TODO: Test get_latlng method
# case insensitive search
get_latlng('tokyo')

{'city': 'tokyo',
 'latitude': np.float64(35.6895266),
 'longitude': np.float64(139.6916809),
 'altitude': np.int64(40)}

### Temperature at latitude and longitude

The following function lookup the weather at the given latitude and longtude.

In [18]:
# TODO: Add tool description

@tool
def get_temperature(latitude: float, longitude: float) -> any:
   """ 
   Get the temperature of the city given by the latitude and longitude. The result is returned as a dictionary.

   Args:
      latitude: latitude of the city
      longitude: longitude of the city

   Returns:
      any: a dictionary with the following keys: temperature_unit, temperature

   Example:
      result = get_temperature(35.6895266, 139.6916809)
   """
   url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m"
   resp = requests.get(url)
   j = resp.json()
   if resp.status_code >= 400:
      raise Exception(j['reason'])
   temperature = j['current']['temperature_2m']
   units = j['current_units']['temperature_2m']
   return { "temperature_unit": units, "temperature": temperature }

In [17]:
# TODO: Test get_temperature method
get_temperature(35.6895266, 139.6916809)

{'temperature_unit': '°C', 'temperature': 28.0}

In [19]:
# TODO: Create a web search tool

def web_search(query: str) -> any:

   search = DuckDuckGoSearchRun()
   return search.invoke(query)

In [21]:
# TODO: Test the web search
web_search("When is the next run of NUS ISS course 'Custom Generative AI Systems for Enterprises'?")

/opt/tmp/ellm-2025-aug11/.venv/lib/python3.12/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


'Jan 13, 2025 · 写在最后：NEXT给我的是惊艳，尤其是4.2和NEXT互换双持的过程中，切实的体会到了NEXT的种种好，比如无比的流畅，非线性的动画等等，这 … 如何评价网易云音乐和网易邮箱大师从鸿蒙 NEXT 应用商店的 "即将到来" 板块中消失? 网易云音乐先是上架了一个非常简陋的, 完全不能用的版本, 连听自己收藏的 … Nov 1, 2024 · 鸿蒙5.0和鸿蒙next的区别鸿蒙5.0和鸿蒙Next是华为推出的两个重要的操作系统版本，它们之间存在一些显著的差异。首先，鸿蒙5.0是鸿蒙操作系 … 目前鸿蒙next的装机量已经达到1000万以上，期待大版本更新的鸿蒙6.0，期待国产操作系统装机量超过1亿，甚至10亿的那天。 知乎，中文互联网高质量的问答社区和创作者聚集的原创内容平台，于 2011 年 1 月正式上线，以「让人们更好的分享知识、经验和见解，找到自己的解答」为品 …'

### Query relational database

The following function queries a relational database (SQLite) view called `album_track`. The table's schema is as follows:
| Field name  | Type          |
|-------------|---------------| 
| AlbumId     | integer       |
| Title       | nvarchar(160) |
| track_name  | nvarchar(200) |
| artist_name | nvarchar(120) |
| duration    | integer       |
| composer    | nvarchar(220) |


In [22]:
# TODO: Add tool description

@tool
def query_album_track(query: str) -> str:
   """ 
   Perform SQL query on the album_track table. Return the result as an array of records. 
   The table name is album_track and has the following columns:
      AlbumId: integer
      Title: nvarchar(160)
      track_name: nvarchar(200)
      artist_name: nvarchar(120)
      duration: integer
      composer: nvarchar(220)
   The duration is in milliseconds.

   Args:
      query: a Valid SQL query in SQLite dialect. The query should only use the select SQL verb.

   Returns:
      str: list of tuple. Each element corresponds to a record from the query

   Example:
      results = query_album_track("select * from album_track where artist_name like '%jackson%'")
   """
   database = "data/chinook_sqlite.sqlite" 
   conn = sqlite3.connect(database)
   try:
      cursor = conn.cursor() 
      rows = cursor.execute(query)
      return rows.fetchall()
   finally:
      conn.close()

In [24]:
# TODO: Test the query_album_track function
query_album_track('select count(*) as rec_count from album_track')

[(3503,)]

### Tools with states

The following isn an example of a more complex tool that requires initialisation

In [25]:
class SQLiteTool(Tool):

   # Required metadata
   name = "chinook_sqlite"

   description = """ 
      Perform SQL query on the album_track table. Return the result as an array of records. 
      The table name is album_track and has the following columns:
         AlbumId: integer
         Title: nvarchar(160)
         track_name: nvarchar(200)
         artist_name: nvarchar(120)
         duration: integer
         composer: nvarchar(220)
      The duration is in milliseconds.
   """

   inputs = {
      "query": {
         "type": "string",
         "description": "A valid SQL query in SQLite dialect. The query should only use the select SQL verb."
      }
   }

   output_type = "string"

   def __init__(self, db_file):
      self.db_file = db_file 
      self.setup()

   def forward(self, query: str) -> str:
      conn = sqlite3.connect(self.db_file)
      try:
         cursor = conn.cursor() 
         rows = cursor.execute(query)
         return rows.fetchall()
      finally:
         conn.close()   